In [1]:
# 패키지 import
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# 데이터셋과 데이터 로더 작성

In [2]:
# 1.3절에서 작성한 클래스를 utils 폴더의 dataloader_image_classification.py에 기재하여 사용합니다
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 개미와 벌의 이미지 파일 경로 리스트를 작성
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Dataset을 만든다
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoader를 만든다
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 사전 객체에 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


# 네트워크 모델 작성

In [3]:
# 학습된 VGG-16 모델을 로드

# VGG-16 모델의 인스턴스를 생성
use_pretrained = True  # 학습된 파라미터를 사용
net = models.vgg16(pretrained=use_pretrained)

# VGG16의 마지막 출력층의 출력 유닛을 개미와 벌의 2개로 바꾸기
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 훈련 모드로 설정
net.train()

print('네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다')


네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다


# 손실 함수 정의

In [4]:
criterion = nn.CrossEntropyLoss()

# 최적화 방법 설정

In [5]:
# 파인 튜닝으로 학습할 파라미터를 params_to_update 변수의 1~3에 저장한다

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 층의 파라미터명을 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 파라미터를 각각 리스트에 저장
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장: ", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장: ", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장: ", name)

    else:
        param.requires_grad = False
        print("경사 계산없음. 학습하지 않음: ", name)


params_to_update_1에 저장:  features.0.weight
params_to_update_1에 저장:  features.0.bias
params_to_update_1에 저장:  features.2.weight
params_to_update_1에 저장:  features.2.bias
params_to_update_1에 저장:  features.5.weight
params_to_update_1에 저장:  features.5.bias
params_to_update_1에 저장:  features.7.weight
params_to_update_1에 저장:  features.7.bias
params_to_update_1에 저장:  features.10.weight
params_to_update_1에 저장:  features.10.bias
params_to_update_1에 저장:  features.12.weight
params_to_update_1에 저장:  features.12.bias
params_to_update_1에 저장:  features.14.weight
params_to_update_1에 저장:  features.14.bias
params_to_update_1에 저장:  features.17.weight
params_to_update_1에 저장:  features.17.bias
params_to_update_1에 저장:  features.19.weight
params_to_update_1에 저장:  features.19.bias
params_to_update_1에 저장:  features.21.weight
params_to_update_1에 저장:  features.21.bias
params_to_update_1에 저장:  features.24.weight
params_to_update_1에 저장:  features.24.bias
params_to_update_1에 저장:  features.26.weight
params_to_update_1

In [6]:

optimizer = optim.SGD([
    {"params":params_to_update_1, "lr":1e-4},
    {"params":params_to_update_2, "lr":5e-4},
    {"params":params_to_update_3, "lr":1e-3},
],momentum=0.9)

# 학습 및 검증 실시

In [7]:
# 모델을 학습시키는 함수를 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 초기 설정
    # GPU가 사용 가능한지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # epoch 루프
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련 모드로
            else:
                net.eval()   # 모델을 검증 모드로

            epoch_loss = 0.0  # epoch 손실의 합
            epoch_corrects = 0  # epoch 정답수

            # 미학습시의 검증 성능을 확인하기 위해 epoch=0의 훈련은 생략
            if (epoch == 0) and (phase == 'train'):
                continue

            # 데이터 로더에서 미니 배치를 꺼내 루프
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPU가 사용 가능하면 GPU에 데이터 보내기
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizer를 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 손실 계산
                    _, preds = torch.max(outputs, 1)  # 라벨 예측

                    # 훈련시에는 오차 역전파법
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0)  # loss의 합계를 갱신
                    # 정답 수의 합계를 갱신
                    epoch_corrects += torch.sum(preds == labels.data)

            # epoch별 loss와 정답률을 표시
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


In [9]:
num_epochs = 30
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

사용 장치:  cuda:0
Epoch 1/30
-------------


100%|██████████| 5/5 [00:01<00:00,  2.72it/s]


val Loss: 0.1668 Acc: 0.9608
Epoch 2/30
-------------


100%|██████████| 8/8 [00:03<00:00,  2.19it/s]


train Loss: 0.1560 Acc: 0.9424


...

val Loss: 0.1253 Acc: 0.9542
Epoch 30/30
-------------


100%|██████████| 8/8 [00:04<00:00,  1.95it/s]


train Loss: 0.0164 Acc: 0.9959


100%|██████████| 5/5 [00:01<00:00,  2.60it/s]

val Loss: 0.1270 Acc: 0.9542


# 학습한 네트워크 저장 및 로드

In [10]:
save_path = "./weights_fine_tuning.pth"
torch.save(net.state_dict(), save_path)

# 파이토치 네트워크 파라미터 로드

In [11]:
load_path = "./weights_fine_tuning.pth"
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

load_weights = torch.load(load_path, map_location={"cuda:0":"cpu"})
net.load_state_dict(load_weights)

<All keys matched successfully>